In [1]:
import re
import pandas as pd
from collections import Counter
import json
import urllib
import requests
from tqdm import tqdm
import geocoder

In [2]:
def clean_adress(address):
    address = re.sub('^\d+,\s+','',address)
    replace_dict = {
        ' г ':'город ',' д ':' дом ',
        ' ул ':'улица ',' пл ':'площадь ',
        ' ДОМ':'дом ','пр-кт':'проспект ',
        'б-р':'бульвар ','р-н':'район ',
        'обл,':'область,','ст-ца':' станица ',
        ' стр ':' строение ',' им ':'имени ',
        ' к ':' корпус ',' им. ':'имени',
        ' г. ':'город ',' ул. ':' улица ',
        'Респ ':'Республика ',' рп ':'рабочий поселок '
    }
    for key in replace_dict.keys():
        address = address.replace(key,replace_dict[key])
    return re.sub('\s{2,}',' ',address)

In [ ]:
xl = pd.ExcelFile('постаматы-ВСП-ПВЗ на 01.12.2022.xlsx')
sheet_names = xl.sheet_names

In [ ]:
df = xl.parse(sheet_names[1], skiprows=1)

In [ ]:
df = df.fillna('НЕТ')

In [ ]:
df.columns = ['Номер постамата','Дата ввода в эксплуатацию',
              'Адрес постамата','Режим работы', 'Аренда']
df['Адрес постамата'] = df['Адрес постамата'].apply(lambda x: clean_adress(x))

In [ ]:
df.columns = ['Номер ВСП/ПВЗ','Дата открытия ВСП/ПВЗ','Адрес ВСП/ПВЗ', 'Режим работы ВСП/ПВЗ']
df['Адрес ВСП/ПВЗ'] = df['Адрес ВСП/ПВЗ'].apply(lambda x: clean_adress(x))

In [ ]:
df.to_csv('VSP.csv',sep='|', index=False)

In [ ]:
df_csv = pd.read_csv('VSP.csv',sep='|')

In [ ]:
pickpoint = pd.read_excel('postamats.xls')

In [ ]:
pickpoint = pickpoint[['ID точки',
                  'Город','Область',
                  'Тип Пункта',
                  'Название пункта приема/выдачи','Адрес пункта приема/выдачи','Время работы','Широта','Долгота']]

In [ ]:
pickpoint['Город'] = pickpoint['Город'].apply(lambda x: f'город {x},')
pickpoint['Область'] = pickpoint['Область'].apply(lambda x: f'{x},')
pickpoint['Адрес пункта приема/выдачи'] = pickpoint['Адрес пункта приема/выдачи'].apply(lambda x: re.sub('^\d+,','',x))
pickpoint['Адрес'] = pickpoint['Город'] + pickpoint['Область'] + pickpoint['Адрес пункта приема/выдачи']
columns = ['ID точки','Адрес','Тип Пункта','Название пункта приема/выдачи','Время работы','Широта','Долгота']
pickpoint = pickpoint[columns]
#pickpoint = pickpoint.drop(columns=['Город','Область','Адрес пункта приема/выдачи'])

In [ ]:
pickpoint.head()

In [ ]:
pickpoint.to_csv('Pickpoint.csv',sep='|',index=False)

In [3]:
import pandas as pd

In [4]:
vsps= pd.read_csv('VSP.csv',sep='|')

In [5]:
vsps.head()

,Номер ВСП/ПВЗ,Дата открытия ВСП/ПВЗ,Адрес ВСП/ПВЗ,Режим работы ВСП/ПВЗ
0,8592/01,2020-09-28 03:14:26,"г Белгород,улица Князя Трубецкого, дом 24","ПН-ПТ 09:00-17:30, СБ 09:00-13:30, ВС выходной"
1,8592/010,2021-02-01 14:11:51,"г Белгород,улица Некрасова, дом 5Б/8","ПН-ПТ 09:00-18:00, СБ 10:00-13:30, ВС выходной"
2,8592/0100,2020-09-25 10:59:46,"Белгородская область,город Алексеевка, пер 1-й...","ПН-ПТ 08:00-17:00, СБ 08:00-13:30, ВС выходной"
3,8592/012,2021-02-01 14:11:51,"г Белгород,улица Железнякова, дом 18А","ПН-ПТ 09:00-18:00, СБ 09:00-13:00, ВС выходной"
4,8592/0127,2021-02-24 17:08:57,"Белгородская область,город Бирюч,улица Карла М...","ПН-ПТ 08:00-16:30, СБ 08:30-13:30, ВС выходной"


In [6]:
vsps.shape

(5350, 4)

In [15]:
len(vsps['Адрес ВСП/ПВЗ'].unique())

5348

In [8]:
sp =  pd.read_excel('Second_page.xlsx')

In [9]:
sp.head()

,Точность,Широта,Долгота,Адрес
0,100.00,50.59265,36.59621,"г Белгород,улица Князя Трубецкого, дом 24"
1,98.62,50.61572,36.58259,"г Белгород,улица Некрасова, дом 5Б/8"
2,96.08,50.62789,38.68997,"Белгородская область,город Алексеевка, пер 1-й..."
3,100.00,50.63256,36.57629,"г Белгород,улица Железнякова, дом 18А"
4,95.92,50.65026,38.40225,"Белгородская область,город Бирюч,улица Карла М..."


In [10]:
sp.shape

(5350, 4)

In [11]:
res_df = pd.merge(vsps, sp, left_on='Адрес ВСП/ПВЗ', right_on='Адрес')

In [12]:
res_df.head()

,Номер ВСП/ПВЗ,Дата открытия ВСП/ПВЗ,Адрес ВСП/ПВЗ,Режим работы ВСП/ПВЗ,Точность,Широта,Долгота,Адрес
0,8592/01,2020-09-28 03:14:26,"г Белгород,улица Князя Трубецкого, дом 24","ПН-ПТ 09:00-17:30, СБ 09:00-13:30, ВС выходной",100.00,50.59265,36.59621,"г Белгород,улица Князя Трубецкого, дом 24"
1,8592/010,2021-02-01 14:11:51,"г Белгород,улица Некрасова, дом 5Б/8","ПН-ПТ 09:00-18:00, СБ 10:00-13:30, ВС выходной",98.62,50.61572,36.58259,"г Белгород,улица Некрасова, дом 5Б/8"
2,8592/0100,2020-09-25 10:59:46,"Белгородская область,город Алексеевка, пер 1-й...","ПН-ПТ 08:00-17:00, СБ 08:00-13:30, ВС выходной",96.08,50.62789,38.68997,"Белгородская область,город Алексеевка, пер 1-й..."
3,8592/012,2021-02-01 14:11:51,"г Белгород,улица Железнякова, дом 18А","ПН-ПТ 09:00-18:00, СБ 09:00-13:00, ВС выходной",100.00,50.63256,36.57629,"г Белгород,улица Железнякова, дом 18А"
4,8592/0127,2021-02-24 17:08:57,"Белгородская область,город Бирюч,улица Карла М...","ПН-ПТ 08:00-16:30, СБ 08:30-13:30, ВС выходной",95.92,50.65026,38.40225,"Белгородская область,город Бирюч,улица Карла М..."


In [17]:
res_df.shape

(5354, 7)

In [16]:
res_df = res_df.drop(columns= ['Адрес ВСП/ПВЗ'])

In [18]:
res_df.to_csv('VSPS.csv', sep='|', index=False)